In [10]:
from NER_cadec import training_loop
model = training_loop("./datasets/cadec", verbose=True, pos_weight=17)

Batch loss 0.098366878926754
Batch loss 0.10075891762971878
Batch loss 0.11353789269924164
Batch loss 0.09963404387235641
Batch loss 0.07758814841508865
Batch loss 0.1001545861363411
Batch loss 0.09876330941915512
Batch loss 0.09366847574710846
Batch loss 0.11208274215459824
Batch loss 0.09293252229690552
Batch loss 0.08346154540777206
Batch loss 0.08818788081407547
Batch loss 0.08771565556526184
Batch loss 0.07849792391061783
Batch loss 0.0818762332201004
Batch loss 0.08885709941387177
Batch loss 0.07698898762464523
Batch loss 0.08449578285217285
Batch loss 0.07460876554250717
Batch loss 0.08292275667190552
Epoch 1, Loss: 0.0907550074160099
Batch loss 0.07997173070907593
Batch loss 0.07845009118318558
Batch loss 0.0729614719748497
Batch loss 0.06558797508478165
Batch loss 0.08648102730512619
Batch loss 0.07118694484233856
Batch loss 0.06716682761907578
Batch loss 0.07694731652736664
Batch loss 0.08007873594760895
Batch loss 0.07001463323831558
Batch loss 0.0652688667178154
Batch loss 

In [11]:
import torch
torch.save(model.state_dict(), "./models/cadec/model_17.pth")

In [13]:
from NER_cadec import evaluation_loop
evaluation_loop(model, "./datasets/cadec")

(0.20902511707109409, 0.6680272108843538, 0.31841763942931256)